<a href="https://colab.research.google.com/github/pedroigp/Fase3-Techchallenge/blob/main/03_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-3u0ktckr/unsloth_4284efd62cb64ed6ad06557b3cf1d10e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-3u0ktckr/unsloth_4284efd62cb64ed6ad06557b3cf1d10e
  Resolved https://github.com/unslothai/unsloth.git to commit 5df2a0ce2a63a8b206c2e857bb44f4f9247610f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/f3-tech-challenge/classified_sentiments.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/f3-tech-challenge/classified_sentiments_reformatado.json"
COMENTARIOS_NOVOS_PATH = "/content/drive/MyDrive/f3-tech-challenge/formatted_dataset_chat_data_test.json"
MODEL_PATH = "/content/drive/MyDrive/f3-tech-challenge/lora_model"

INSTRUCAO = "Classify the sentiment of the following comment as positive, negative, or neutral. No need to explain."

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    with open(data, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for item in data:
        instructions.append(INSTRUCAO)
        inputs.append(item['content'])
        outputs.append(item['sentiment'])

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")



In [6]:
format_dataset_into_model_input(DATA_PATH)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Dataset salvo em /content/drive/MyDrive/f3-tech-challenge/classified_sentiments_reformatado.json
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Classify the sentiment of the following comment as positive, negative, or neutral. No need to explain.",
        "I am a very religious person and like to learn about the different ones out there so when I came across this book I had to get it to learn more. I learned a lot after reading this book it has a lot of stories about what had happened in the biblical era that we might not have heard of since they were rejected from the bible.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the sentiment of the following comment as positive, negative, or neutral. No need to explain.

### Input:
I am a very religious person and like to learn about the different ones out there so when I came across this book I had to get it to learn more. I learned a lot after reading this book it has a lot of stories about what had happened in the biblical era that we might not have heard of since they were rejected from the bible.

### Response:
This comment is positive. It's a positive review of a book about the Bible, which is a very popular topic.<|end_of_text|>


In [10]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pedroigp (pedroigp-personal-use) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.192600
2,3.173800
3,3.156700
4,2.958600
5,2.985200
6,2.838500
7,2.572000
8,2.160900
9,1.904600
10,2.115300


In [13]:
model.save_pretrained(MODEL_PATH) # Local saving
tokenizer.save_pretrained(MODEL_PATH)

('/content/drive/MyDrive/f3-tech-challenge/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/f3-tech-challenge/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/f3-tech-challenge/lora_model/tokenizer.json')

In [20]:
#import json
#comments_test = []

#with open(COMENTARIOS_NOVOS_PATH, 'r') as news_file:
#  comments_test = json.loads(news_file.read())['comments']

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/f3-tech-challenge/fazerarquivo.json'

In [14]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        #model_name = "/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        model_name = MODEL_PATH,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [15]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "Classify the sentiment of the following comment as positive, negative, or neutral. No need to explain",
        #comments_test[1],
        "I am a very religious person and like to learn about the different ones out there so when I came across this book I had to get it to learn more. I learned a lot after reading this book it has a lot of stories about what had happened in the biblical era that we might not have heard of since they were rejected from the bible.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the sentiment of the following comment as positive, negative, or neutral. No need to explain

### Input:
I am a very religious person and like to learn about the different ones out there so when I came across this book I had to get it to learn more. I learned a lot after reading this book it has a lot of stories about what had happened in the biblical era that we might not have heard of since they were rejected from the bible.

### Response:
Positive<|end_of_text|>
